In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime,timedelta
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('mc1-reports-data.csv')
df.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1


In [7]:
feature_list = ["sewer_and_water","power", "roads_and_bridges", "medical", "buildings","shake_intensity"]
cols = ['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity','location'] 

In [4]:
for i in range(len(feature_list)):
    feature = feature_list[i]
    print("{} has NAN: {}".format(feature,df[feature].isnull().sum()))

sewer_and_water has NAN: 171
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 47441
buildings has NAN: 170
shake_intensity has NAN: 12144


In [5]:
regionGroup = df.groupby("location") 

In [6]:
#print out for finding the missing data points
for region_num in range(1,20):
    region_df = regionGroup.get_group(region_num)
    print('in region {}:'.format(region_num))
    for i in range(len(feature_list)):
        feature = feature_list[i]
        print("{} has NAN: {}".format(feature,region_df[feature].isnull().sum()))
    print('-----------------------')

in region 1:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 889
-----------------------
in region 2:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 5294
buildings has NAN: 0
shake_intensity has NAN: 974
-----------------------
in region 3:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 770
-----------------------
in region 4:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 2867
buildings has NAN: 0
shake_intensity has NAN: 225
-----------------------
in region 5:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has NAN: 0
buildings has NAN: 0
shake_intensity has NAN: 374
-----------------------
in region 6:
sewer_and_water has NAN: 0
power has NAN: 0
roads_and_bridges has NAN: 0
medical has N

In [8]:
starttimelist = []
time_current = pd.to_datetime("2020-04-06 00:00:00")
last_starttime = pd.to_datetime("2020-04-10 14:00:00")
while (time_current <= last_starttime):
    starttimelist.append(time_current)
    time_current += timedelta(minutes=30)

##create timelist: elements are timestamp
timelist = []
time_current = pd.to_datetime("2020-04-06 00:00:00")
time_end = pd.to_datetime("2020-04-11 00:01:00")
##(time_start - time_end).total_seconds() < 0
while (time_end -time_current).total_seconds() > 0:
    timelist.append(time_current)
    time_current += timedelta(minutes=30)

region_dict = {(num+1): None for num in range(19)}

In [9]:
def convert_time_interval(time):
    approx = round(time.minute//30.0) * 30
    time = time.replace(minute=0)
    time += timedelta(seconds=approx * 60)
    return time

def convert_time_region(region_df):
    for idx, row in region_df.iterrows():
        current_time = row['time']
        time_interval = convert_time_interval(current_time)
        region_df.loc[idx,'time'] = time_interval
    return region_df

#function for sorting timeline of a region
def sort_regioni(i):
    """
    parameter: int i is referred to the region No.
    """
    region = regionGroup.get_group(i)
    region['time'] = pd.to_datetime(region.time)
    region = region.sort_values(by='time').reset_index().drop(labels=['index'], axis=1)
    region = convert_time_region(region)
    return region
    

In [10]:
for i in range(1,20):
    region_dict[i] = sort_regioni(i)

/Users/zhangyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
cols_w_time = ['time','sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity','location'] 
tenhour_df = pd.DataFrame(columns=cols_w_time)
data_tenhour = []
for region_num in range(1,20):
    region_df = region_dict[region_num]
    current_first_starttime = pd.to_datetime("2020-04-06 00:00:00")
    current_last_starttime = None
    
    for idx, row in region_df.iterrows():
        if row['time'] - current_first_starttime >= timedelta(hours=10):
            num_of_intervals = math.ceil((row['time'] - current_first_starttime - timedelta(hours=10)).total_seconds()/60/30)
            current_first_starttime += timedelta(minutes = 30*num_of_intervals)
        if row['time']  < last_starttime:
            current_last_starttime = current_first_starttime
        else:
            current_last_starttime = row['time']
            
        current_starttime = current_first_starttime
        while(current_starttime <= current_last_starttime):
            row_modified = row.copy(deep=True)
            row_modified['time'] = current_starttime
           # print(row_modified)
            data_tenhour.append(row_modified.tolist())
           # print('added')
            current_starttime += timedelta(minutes=30)

tenhour_df = pd.DataFrame(columns=cols_w_time, data=data_tenhour)

In [12]:
tenhour_data_dict = {num: pd.DataFrame(columns=cols,index= starttimelist) for num in range(1,20)}
tenhour_regionGroup = tenhour_df.groupby('location')
for i in range(1,20):
    region_df = tenhour_regionGroup.get_group(i)
    current_df = tenhour_data_dict[i]
    timegroup_mean_df = region_df.groupby('time').mean()
    timegroup_mean_df
    for index,row in timegroup_mean_df.iterrows():
        current_df.loc[index] = row
    current_df['location'] = current_df['location'].fillna(i)
    current_df = current_df.fillna(-1).reset_index()
    tenhour_data_dict[i] = current_df

In [13]:
merge_10h_i_df = pd.concat([tenhour_data_dict[i] for i in range(1,20)])
merge_10h_i_df['location'] = merge_10h_i_df['location'].astype(int)

In [14]:
merge_10h_i_df.to_csv('merge_10h_i.csv')

,index,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-06 00:00:00,5.636364,4.848485,4.878788,6.030303,6.151515,0.5,1
1,2020-04-06 00:30:00,2.000000,9.000000,9.000000,6.000000,0.000000,-1.0,1
2,2020-04-06 01:00:00,1.000000,4.000000,4.000000,8.000000,8.000000,0.0,1
3,2020-04-06 01:30:00,8.000000,8.666667,8.000000,3.333333,5.333333,1.0,1
4,2020-04-06 02:00:00,4.000000,2.666667,5.333333,7.000000,6.000000,-1.0,1
...,...,...,...,...,...,...,...,...
234,2020-04-10 21:00:00,5.625000,4.500000,6.500000,-1.000000,6.250000,1.0,19
235,2020-04-10 21:30:00,6.000000,4.285714,6.142857,-1.000000,6.285714,1.0,19
236,2020-04-10 22:00:00,5.400000,5.200000,6.600000,-1.000000,5.600000,1.0,19
237,2020-04-10 22:30:00,5.000000,5.000000,8.250000,-1.000000,4.500000,1.0,19
